<a href="https://colab.research.google.com/github/lemart0/Data_Tesina/blob/main/An%C3%A1lisis_de_Datos_Andrea_Boettner_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Análisis de datos**
Este archivo contiene los análisis estadísticos inferenciales



#**Montando el archivo con los datos de la tesis**

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#Accediendo al archivo con los datos de interés y creando un dataframe de pandas para trabajar con la información

ruta ='/content/drive/My Drive/Tesis_andy_data/datos.csv'
df = pd.read_csv(ruta)
#cargado

In [7]:
#Chequeo de que haya cargado bien el archivo
df.columns


Index(['Unnamed: 0', 'ID', 'tb_score', 'tb_errors', 'tb_meanrt_correct',
       'tb_meanrt_incorrect', 'ne_score', 'ne_errors', 'ne_meanrt_correct',
       'ne_meanrt_incorrect',
       ...
       'dn_tokens_pal', 'dn_lemmas_pal', 'dmlu_pal', 'dttr_pal', 'dmsttr_pal',
       'dmattr_pal', 'dhdd_pal', 'dmaas_pal', 'dmtld_pal', 'dmtldma_pal'],
      dtype='object', length=207)

In [8]:
# Información general del dataframe
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Columns: 207 entries, Unnamed: 0 to dmtldma_pal
dtypes: float64(192), int64(10), object(5)
memory usage: 111.7+ KB


#**Análisis correlacionales**


**VARIABLES DE INTERÉS POR OBJETIVOS:**

#**Objetivo específico 2:**

**a. Autorregulación** -> medida por tb_score (Subtest del NeuroUX) que evalúa control inhibitorio y por Esfuerzo.de.control (Subtest del CQB-)

**b. Funcionamiento ejecutivo** -> medido por "PA" y"CO"  (Subtest del Woodcock-Muñoz), "ne_score" (Subtest del NeuroUX).



#**Objetivo específico 3:**

**a. Características sociodemográficas de cuidadores y entorno familiar medidas por** -> 'Educación_de_la_madre',
    'Educación_del_padre',
    'Libros_en_el_hogar',
    'Frecuencia_de_lectura_compartida',
    'Materiales_didácticos_en_el_hogar',
    'Percepción_de_ingresos',
    'Percepción_de_recursos',
    'Percepción_de_calma',
    'Usa_dispositivos_electrónicos_cotidianamente'

**b. Funcionamiento ejecutivo y habiliades cognitivas** -> medido por "PA" y "CO" (Subtest del Woodcock-Muñoz) "ne_score" 'mt_score', 'bcd_score' (Subtest del NeuroUX)

#**Objetivo específico 4:**
**a. Características sociodemográficas de cuidadores y entorno familiar medidas por** -> 'Educación_de_la_madre',
    'Educación_del_padre',
    'Libros_en_el_hogar',
    'Frecuencia_de_lectura_compartida',
    'Materiales_didácticos_en_el_hogar',
    'Percepción_de_ingresos',
    'Percepción_de_recursos',
    'Percepción_de_calma',
    'Usa_dispositivos_electrónicos_cotidianamente'

**b. Control emocional y temperamento infantil** -> medida por tb_score (Subtest del NeuroUX) que evalúa control inhibitorio y por 'Esfuerzo.de.control', 'Extraversión' y 'Afectividad.negativa' (Subtest del CQB)

Se realiza una matriz de correlación. Dado de que las variables sociodemográficas a correlacionar son de tipo ordinales se utilizó la correlación de Spearman.

In [9]:
# Variables cognitivas, atencionales y de funciones ejecutivas
variables_test = [
    'PA',                       # Subtest Problemas Aplicados (Woodcock-Muñoz)
    'CO',                       # Subtest Comprensión de Oraciones (Woodcock-Muñoz)
    'tb_score',                 # Toca Botón (NeuroUX)
    'ne_score',                 # No Encaja (NeuroUX)
    'mt_score',                 # Juego de Memoria (NeuroUX)
    'bcd_score',                # Bloques de Corsi (NeuroUX)
    'CHAOS',                    # Entorno familiar (CHAOS)
    'Extraversión',             # Subescala CBQ
    'Esfuerzo.de.control',      # Subescala CBQ
    'Afectividad.negativa'      # Subescala CBQ
]

# Variables sociodemográficas cuantificadas
variables_socio = [
    'Educación_de_la_madre',
    'Educación_del_padre',
    'Libros_en_el_hogar',
    'Frecuencia_de_lectura_compartida',
    'Materiales_didácticos_en_el_hogar',
    'Percepción_de_ingresos',
    'Percepción_de_recursos',
    'Percepción_de_calma',
    'Usa_dispositivos_electrónicos_cotidianamente',
]

# Lista completa
vars_todas = variables_test + variables_socio

In [11]:
# Subconjunto con las columnas de interés
subset = df[vars_todas]  # Asegurate de que df_completo tenga todo unido por ID

In [16]:
import numpy as np
from scipy.stats import spearmanr

def matriz_spearman_con_p(dataframe):
    cols = dataframe.columns
    rho  = pd.DataFrame(index=cols, columns=cols, dtype=float)
    pval = pd.DataFrame(index=cols, columns=cols, dtype=float)

    for i in cols:
        for j in cols:
            pares = dataframe[[i, j]].dropna()   #elimno filas con NaN ya que sino no podría realizar la correlación
            if len(pares) >= 3:
                r_obj = spearmanr(pares[i], pares[j])
                # Extraer si devuelve matriz (array)
                r = r_obj.correlation if hasattr(r_obj, 'correlation') else r_obj[0]
                p = r_obj.pvalue if hasattr(r_obj, 'pvalue') else r_obj[1]
            else:
                r, p = np.nan, np.nan
            rho.loc[i, j]  = round(r, 2) if np.isscalar(r) else np.nan   #r recondeado a 2 decimales
            pval.loc[i, j] = round(p, 4) if np.isscalar(p) else np.nan   #p redondeado a 4 decimales

    return rho, pval

In [17]:
# Mostrando los resultados del análisis con el valor de r y el p valor correspondiente.
matriz_r, matriz_p = matriz_spearman_con_p(subset)

# Mostrás los resultados
print("📌 Matriz de correlaciones de Spearman (ρ)")
display(matriz_r)

print("📌 Matriz de p-valores correspondientes")
display(matriz_p)

📌 Matriz de correlaciones de Spearman (ρ)


,PA,CO,tb_score,ne_score,mt_score,bcd_score,CHAOS,Extraversión,Esfuerzo.de.control,Afectividad.negativa,Educación_de_la_madre,Educación_del_padre,Libros_en_el_hogar,Frecuencia_de_lectura_compartida,Materiales_didácticos_en_el_hogar,Percepción_de_ingresos,Percepción_de_recursos,Percepción_de_calma,Usa_dispositivos_electrónicos_cotidianamente
PA,NaN,0.73,0.36,0.64,0.44,0.58,0.08,-0.07,0.27,0.02,0.23,-0.07,0.24,0.20,0.24,-0.16,-0.19,-0.02,-0.00
CO,0.73,NaN,0.34,0.67,0.60,0.65,0.06,-0.17,0.15,0.09,0.22,0.05,0.20,0.09,0.23,-0.07,-0.05,0.06,0.08
tb_score,0.36,0.34,NaN,0.47,0.22,0.70,-0.21,-0.33,-0.01,0.01,0.17,-0.03,0.50,0.53,0.41,0.03,0.02,0.15,-0.02
ne_score,0.64,0.67,0.47,NaN,0.46,0.67,-0.21,-0.51,0.29,-0.03,0.22,0.23,0.02,0.15,0.28,0.21,0.05,0.20,0.14
mt_score,0.44,0.60,0.22,0.46,NaN,0.41,0.13,-0.05,0.24,-0.05,0.00,0.01,0.05,0.13,0.26,-0.15,0.02,0.08,0.28
bcd_score,0.58,0.65,0.70,0.67,0.41,NaN,0.02,-0.63,0.20,0.13,0.40,-0.11,0.36,0.46,0.19,0.11,0.34,0.45,0.02
CHAOS,0.08,0.06,-0.21,-0.21,0.13,0.02,NaN,-0.06,-0.05,0.42,-0.03,0.01,0.01,0.01,-0.27,-0.18,0.10,-0.19,-0.05
Extraversión,-0.07,-0.17,-0.33,-0.51,-0.05,-0.63,-0.06,NaN,0.03,0.16,0.03,-0.11,0.13,0.16,0.21,0.13,0.03,-0.13,-0.17
Esfuerzo.de.control,0.27,0.15,-0.01,0.29,0.24,0.20,-0.05,0.03,NaN,0.18,0.06,-0.05,0.25,0.32,0.03,-0.16,-0.23,-0.01,0.04
Afectividad.negativa,0.02,0.09,0.01,-0.03,-0.05,0.13,0.42,0.16,0.18,NaN,-0.01,0.05,0.16,-0.03,-0.08,-0.01,-0.07,-0.12,0.19


📌 Matriz de p-valores correspondientes


,PA,CO,tb_score,ne_score,mt_score,bcd_score,CHAOS,Extraversión,Esfuerzo.de.control,Afectividad.negativa,Educación_de_la_madre,Educación_del_padre,Libros_en_el_hogar,Frecuencia_de_lectura_compartida,Materiales_didácticos_en_el_hogar,Percepción_de_ingresos,Percepción_de_recursos,Percepción_de_calma,Usa_dispositivos_electrónicos_cotidianamente
PA,NaN,0.0000,0.1099,0.0017,0.0445,0.0059,0.5156,0.5640,0.0228,0.8864,0.0666,0.5778,0.0595,0.1041,0.0534,0.1998,0.1274,0.8833,0.9739
CO,0.0000,NaN,0.1273,0.0008,0.0037,0.0015,0.6326,0.1700,0.2210,0.4567,0.0854,0.7227,0.1188,0.4884,0.0625,0.5568,0.7001,0.6084,0.5281
tb_score,0.1099,0.1273,NaN,0.0303,0.3330,0.0004,0.4058,0.1484,0.9499,0.9490,0.4750,0.9212,0.0252,0.0152,0.0718,0.9067,0.9298,0.5323,0.9322
ne_score,0.0017,0.0008,0.0303,NaN,0.0357,0.0010,0.4108,0.0180,0.2087,0.8939,0.3453,0.3810,0.9173,0.5353,0.2348,0.3800,0.8231,0.4083,0.5502
mt_score,0.0445,0.0037,0.3330,0.0357,NaN,0.0638,0.6047,0.8232,0.2905,0.8227,0.9934,0.9809,0.8299,0.5832,0.2618,0.5181,0.9417,0.7383,0.2251
bcd_score,0.0059,0.0015,0.0004,0.0010,0.0638,NaN,0.9353,0.0021,0.3826,0.5862,0.0797,0.6725,0.1228,0.0393,0.4297,0.6468,0.1412,0.0453,0.9280
CHAOS,0.5156,0.6326,0.4058,0.4108,0.6047,0.9353,NaN,0.6191,0.7015,0.0008,0.8162,0.9147,0.9220,0.9187,0.0359,0.1897,0.4607,0.1389,0.6895
Extraversión,0.5640,0.1700,0.1484,0.0180,0.8232,0.0021,0.6191,NaN,0.7995,0.1870,0.8418,0.4118,0.3180,0.1920,0.0950,0.2895,0.7888,0.2920,0.1776
Esfuerzo.de.control,0.0228,0.2210,0.9499,0.2087,0.2905,0.3826,0.7015,0.7995,NaN,0.1398,0.6453,0.7107,0.0478,0.0097,0.8222,0.1931,0.0606,0.9447,0.7237
Afectividad.negativa,0.8864,0.4567,0.9490,0.8939,0.8227,0.5862,0.0008,0.1870,0.1398,NaN,0.9173,0.7345,0.2074,0.8289,0.5155,0.9390,0.6028,0.3343,0.1197
